In [ ]:
# Queu的几个特性
from queue import Queue
my_queue = Queue()
def consumer():
    """
    此函数用于模拟消费者的动作
    """
    print('Consumer waiting\n')
    my_queue.get()  # 从队列中拿出一个元素
    print(f'Consumer Done')
from threading import Thread
thread = Thread(target=consumer)
thread.start()
# XXX get方法会阻塞所在线程,即使star方法在前
print('Producing putting')
my_queue.put(object())
print('Producing Done')
thread.join()




Consumer waiting



In [10]:
# 限制queue的容量,当元素超限后queue的put方法所在线程会阻塞,直到可以放进去
from queue import Queue
from threading import Thread
import time
my_queue = Queue(2)
def consumer():
    """
    此函数用于模拟消费者的动作
    """
    print('Consumer waiting\n')
    time.sleep(0.1)
    my_queue.get()  # 从队列中拿出一个元素
    print(f'Consumer get 1')

    my_queue.get()  # 从队列中拿出一个元素
    print(f'Consumer get 2')
    print(f'Consumer get done')
thread = Thread(target=consumer)
thread.start()
my_queue.put(object())
print('Producing put 1')
time.sleep(1)
my_queue.put(object())
print('Producing put 2')
print('Producing done')
thread.join()

Consumer waiting
Producing put 1
Producing put 2
Producing done

Consumer get 1
Consumer get 2
Consumer get done


In [15]:
# XXX Queue还可以通过'task_done'和'join'方法实现等待队列元素全部耗尽的目的
# join的释放条件是put次数和task_done执行次数一致
in_queue = Queue()
def consumer():
    print('Consumer waiting\n')
    work = in_queue.get()
    print('Consumer working')
    # doing work
    ...
    print('Counsumer Done')
    time.sleep(1)
    in_queue.task_done()
thread = Thread(target=consumer)
thread.start()
print('Producing putting')
in_queue.put(object())
in_queue.join()
print('Producing Done')
thread.join()

Consumer waiting
Producing putting

Consumer working
Counsumer Done
Producing Done


In [67]:
class CloseableQueue(Queue):
    SENTINEL = object()  # 自定义的标志元素，当其进入队列后作为最后一个元素的标志

    def close(self):
        self.put(self.SENTINEL)
    
    def __iter__(self):
        while True:
            item = self.get()
            try:
                if item is self.SENTINEL:
                    return
                yield item
            finally:
                self.task_done()  # 用于追踪队列进度

class StoppableWorker(Thread):
    """
    自定义线程类,可以链接两个队列,并将结果传入
    """
    def __init__(self, func, in_queue: CloseableQueue, out_queue: CloseableQueue, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.func = func
        self.in_queue = in_queue
        self.out_queue = out_queue
        self.count = 0
        print(f'Thread star func{self.func.__name__}')
    
    def run(self):
        # count = 0
        for item in self.in_queue:
            result = self.func(item)
            self.out_queue.put(result)
            self.count += 1
            # if not self.count // 10:
            #     # time.sleep(0.5)
            #     print(f'{self.func.__name__=}')

In [68]:
def download(item):
    time.sleep(0.01)
    print(f'download doing')
    ...
    # return item
def resize(item):
    time.sleep(0.02)
    print(f'resize doing')
    ...
def upload(item):
    time.sleep(0.04)
    print(f'upload doing')
    ...

In [69]:
# 初始化四个自定义队列
downloda_queue = CloseableQueue()
resize_queue = CloseableQueue()
upload_queue = CloseableQueue()
done_queue = CloseableQueue()
# 建立线程池
threads = [StoppableWorker(download, downloda_queue, resize_queue),
           StoppableWorker(resize, resize_queue, upload_queue),
           StoppableWorker(upload, upload_queue, done_queue),
           ]
# 开启线程池
for thread in threads:
    thread.start()
# 向最上游注入元素
for _ in range(10):
    downloda_queue.put(object())
# 给最上游打入队尾标志
downloda_queue.close()
# 等待download队列的元素处理完后 给相邻的队列打入队尾标志
downloda_queue.join()
resize_queue.close()
resize_queue.join()
upload_queue.close()
upload_queue.join()
print(f'{done_queue.qsize()=}')

for thread in threads:
    thread.join()
"""
注意此模型的线程是平行的, 每个队列在有数据元素时候都会调用处理函数进行操作, 而不是串行的等待
"""

Thread star funcdownload
Thread star funcresize
Thread star funcupload
download doing
download doing
resize doing
download doing
download doing
resize doing
download doing
upload doing
resize doing
download doing
download doing
resize doing
upload doing
download doing
resize doing
download doing
download doing
upload doing
resize doing
resize doing
upload doing
resize doing
resize doing
upload doing
resize doing
upload doing
upload doing
upload doing
upload doing
upload doing
done_queue.qsize()=10


'\n注意此模型的线程是平行的, 每个队列在有数据元素时候都会调用处理函数进行操作, 而不是串行的等待\n'